## PySpark Notes

Filtering Rows

[Really solid examples here](https://sparkbyexamples.com/pyspark/pyspark-where-filter/)

In [1]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
import pandas as pd

In [2]:
spark = SparkSession.builder.appName("Spark Intro").getOrCreate()

#### Read in data

In [3]:
df = spark.read.csv('./data/store.csv', header=True, inferSchema=True)
df.columns

['Date',
 'Customer ID',
 'Customer Name',
 'Segment',
 'Country',
 'City',
 'State',
 'Postal Code',
 'Region',
 'Product ID',
 'Category',
 'Sub-Category',
 'Product Name',
 'Sales',
 'Quantity',
 'Discount',
 'Profit',
 'RATING']

#### Drop a few columns (for simplicity)

In [4]:
df = df.drop('Customer ID', 'State', 'Region', 'Postal Code', 'Product ID', 'Discount')
df.show(n=10)

+----------+---------------+---------+-------------+---------------+---------------+------------+--------------------+--------+--------+--------+------+
|      Date|  Customer Name|  Segment|      Country|           City|       Category|Sub-Category|        Product Name|   Sales|Quantity|  Profit|RATING|
+----------+---------------+---------+-------------+---------------+---------------+------------+--------------------+--------+--------+--------+------+
|2020-09-09|    Claire Gute| Consumer|United States|      Henderson|      Furniture|   Bookcases|Bush Somerset Col...|  261.96|       2| 41.9136|   8.0|
|2020-10-27|    Claire Gute| Consumer|United States|      Henderson|      Furniture|      Chairs|Hon Deluxe Fabric...|  731.94|       3| 219.582|   8.0|
|2020-09-10|Darrin Van Huff|Corporate|United States|    Los Angeles|Office Supplies|      Labels|Self-Adhesive Add...|   14.62|       2|  6.8714|   8.0|
|2020-04-25| Sean O'Donnell| Consumer|United States|Fort Lauderdale|      Furnitur

In [5]:
print(f'There are {df.count()} rows and {len(df.columns)} columns in this dataframe')

There are 9994 rows and 12 columns in this dataframe


#### Find all rows where revenue was greater than $5000 for orders from Los Angeles. How many rows matched this criteria?

In [6]:
df.dtypes

[('Date', 'string'),
 ('Customer Name', 'string'),
 ('Segment', 'string'),
 ('Country', 'string'),
 ('City', 'string'),
 ('Category', 'string'),
 ('Sub-Category', 'string'),
 ('Product Name', 'string'),
 ('Sales', 'string'),
 ('Quantity', 'string'),
 ('Profit', 'double'),
 ('RATING', 'double')]

In [7]:
# First create a revenue column
df = df.withColumn(colName='Revenue', col=df.Sales*df.Quantity)

required_rows = df[(df.Revenue > 5000) & (df.City == 'Los Angeles')]

matches = required_rows.count()
print(f'There are {matches} rows that match this criteria\n')
required_rows.show(n=matches)

There are 38 rows that match this criteria

+----------+-------------------+---------+-------------+-----------+---------------+------------+--------------------+--------+--------+---------+------+------------------+
|      Date|      Customer Name|  Segment|      Country|       City|       Category|Sub-Category|        Product Name|   Sales|Quantity|   Profit|RATING|           Revenue|
+----------+-------------------+---------+-------------+-----------+---------------+------------+--------------------+--------+--------+---------+------+------------------+
|2020-04-07|    Brosina Hoffman| Consumer|United States|Los Angeles|     Technology|      Phones|Mitel 5320 IP Pho...| 907.152|       6|  90.7152|   8.0|          5442.912|
|2020-07-15|    Brosina Hoffman| Consumer|United States|Los Angeles|      Furniture|      Tables|Chromcraft Rectan...|1706.184|       9|  85.3092|  10.0|15355.655999999999|
|2020-11-02|      Jas O'Carroll| Consumer|United States|Los Angeles|      Furniture|      T

#### Filter observations between 2020-04-19 and 2020-11-20 and verify that it was done properly

In [8]:
# Essentially this is df['Date'] = pd.to_datetime(df.Date)
df = df.withColumn(colName='Date', col=F.to_date(df.Date))

In [9]:
start_date = pd.to_datetime('2020-04-19')
end_date = pd.to_datetime('2020-11-20')

df_between_dates = df[df.Date.between(lowerBound=start_date, upperBound=end_date)]
df_between_dates.agg(F.min('date'), F.max('date')).show()

+----------+----------+
| min(date)| max(date)|
+----------+----------+
|2020-04-19|2020-11-20|
+----------+----------+



#### Filter people whose name begins with a 'Z' and are from Salem

In [10]:
df[(df['Customer Name'].startswith('Z')) & (df.City == 'Salem')].show()

+----------+----------------+--------+-------------+-----+---------------+------------+--------------------+------+--------+--------+------+-----------------+
|      Date|   Customer Name| Segment|      Country| City|       Category|Sub-Category|        Product Name| Sales|Quantity|  Profit|RATING|          Revenue|
+----------+----------------+--------+-------------+-----+---------------+------------+--------------------+------+--------+--------+------+-----------------+
|2020-10-07|Zuschuss Carroll|Consumer|United States|Salem|     Technology|      Phones|i.Sound Portable ...|84.784|       2|-20.1362|  10.0|          169.568|
|2020-11-16|Zuschuss Carroll|Consumer|United States|Salem|Office Supplies|       Paper|           Xerox 225|20.736|       4|  7.2576|   8.0|           82.944|
|2020-04-19|Zuschuss Carroll|Consumer|United States|Salem|Office Supplies|     Binders|Clear Mylar Reinf...|16.821|       3|-12.8961|   5.0|50.46300000000001|
|2020-12-11|Zuschuss Carroll|Consumer|United S

#### Find all paper related orders where cost was greater than $4000. Select only sub-category, revenue, cost and profit
Don't use square brackets for the filtering

In [11]:
df = df.withColumn(colName='Cost', col=df.Revenue-df.Profit)

# `filter` and `where` do the same thing
filtered_df = df.filter( (df['Sub-Category'] == 'Paper') & (df['Cost'] > 4000) )

# Selecting
filtered_df.select('Sub-Category', 'Revenue', 'Cost', 'Profit').show()

+------------+-----------------+-----------------+--------+
|Sub-Category|          Revenue|             Cost|  Profit|
+------------+-----------------+-----------------+--------+
|       Paper|5137.650000000001|         4785.354| 352.296|
|       Paper|         4136.384|        4040.3608| 96.0232|
|       Paper|         4188.496|         4087.811| 100.685|
|       Paper|4476.639999999999|4329.550399999999|147.0896|
+------------+-----------------+-----------------+--------+



#### Filter rows where the city was Madison or Fort Lauderdale or West Jordan

In [12]:
df.where(df.City.isin(['Madison', 'Fort Lauderdale', 'West Jordan'])).show()

+----------+-----------------+--------+-------------+---------------+---------------+------------+--------------------+--------+--------+--------+------+------------------+------------------+
|      Date|    Customer Name| Segment|      Country|           City|       Category|Sub-Category|        Product Name|   Sales|Quantity|  Profit|RATING|           Revenue|              Cost|
+----------+-----------------+--------+-------------+---------------+---------------+------------+--------------------+--------+--------+--------+------+------------------+------------------+
|2020-04-25|   Sean O'Donnell|Consumer|United States|Fort Lauderdale|      Furniture|      Tables|Bretford CR4500 S...|957.5775|       5|-383.031|   9.0|         4787.8875|         5170.9185|
|2020-07-29|   Sean O'Donnell|Consumer|United States|Fort Lauderdale|Office Supplies|     Storage|Eldon Fold 'N Rol...|  22.368|       2|  2.5164|   7.0|            44.736|           42.2196|
|2020-02-24|        Pete Kriz|Consumer|U

#### Filter full names ending with 'berg' or containing 'erin' or 'Erin' and is from Philadelphia or Wilmington

In [13]:
c1 = (df['Customer Name'].endswith('berg'))
c2 = (df['Customer Name'].rlike('[Ee]rin'))
c3 = (df.City.rlike('[Pp]hiladelphia|[Ww]ilmington'))
    
df.where((c1 | c2) & c3).show()

+----------+-------------------+---------+-------------+------------+---------------+------------+--------------------+------------------+--------+---------+-------+------------------+------------------+
|      Date|      Customer Name|  Segment|      Country|        City|       Category|Sub-Category|        Product Name|             Sales|Quantity|   Profit| RATING|           Revenue|              Cost|
+----------+-------------------+---------+-------------+------------+---------------+------------+--------------------+------------------+--------+---------+-------+------------------+------------------+
|2020-03-16|Catherine Glotzbach| Consumer|United States|Philadelphia|Office Supplies|       Paper|"Wirebound Messag...| 3 Forms per Page"|  45.056|      0.2|15.2064|              null|              null|
|2020-10-05|Catherine Glotzbach| Consumer|United States|Philadelphia|Office Supplies|     Binders|  JM Magazine Binder|            29.718|       6| -21.7932|    7.0|           178.308|